In [53]:
import pandas as pd
import re
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [54]:
df = pd.read_json('http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_5.json.gz', lines=True)    
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A32T2H8150OJLU,B00000JBLH,ARH,"[3, 4]","I bought my first HP12C in about 1984 or so, a...",5,"A solid performer, and long time friend",1094169600,"09 3, 2004"
1,A3MAFS04ZABRGO,B00000JBLH,"Let it Be ""Alan""","[7, 9]",WHY THIS BELATED REVIEW? I feel very obliged t...,5,"Price of GOLD is up, so don't bury the golden ...",1197676800,"12 15, 2007"
2,A1F1A0QQP2XVH5,B00000JBLH,Mark B,"[3, 3]",I have an HP 48GX that has been kicking for mo...,2,"Good functionality, but not durable like old HPs",1293840000,"01 1, 2011"
3,A49R5DBXXQDE5,B00000JBLH,R. D Johnson,"[7, 8]",I've started doing more finance stuff recently...,5,One of the last of an almost extinct species,1145404800,"04 19, 2006"
4,A2XRMQA6PJ5ZJ8,B00000JBLH,Roger J. Buffington,"[0, 0]",For simple calculations and discounted cash fl...,5,Still the best,1375574400,"08 4, 2013"


In [0]:
def removepunc(l):
    final = ''
    for x in l.split():
        for y in x:
            if y.isalpha():
                final += y
        final += ' '
    return final

df['reviewText']=df['reviewText'].apply(removepunc)

In [0]:
from nltk.tokenize import MWETokenizer 
from nltk import word_tokenize

def mwe(str):
    d = {}
    tk = MWETokenizer() 
    d['MWETokenizer'] = tk.tokenize(str.split())
    d['word_tokenize'] = word_tokenize(str)
    temp = nltk.pos_tag(str.split())
    d['pos_tag'] = temp

    adj = []
    for x in temp:
        if x[1] == 'JJ':
            adj.append(x[0])
    d['adjective'] = adj

    return d

#print(mwe('geeks for geeks g f g'))
df['nltk']=df['reviewText'].apply(mwe)
#df.head()

In [57]:
def goodbadadjectives(d1):
    file = open('bad words.txt')
    d = {}
    l = file.readlines()
    for x in range(len(l)):
        l[x] = l[x][:-1]
    l_adj = d1['adjective']
    negative = 0
    positive = 0
    l_negative = []
    l_positive = []
    for x in l_adj:
        if x in l:
            negative += 1
            l_negative.append(x)
        else:
            positive += 1
            l_positive.append(x)
    d['Total Adjectives'] = positive + negative
    d['Number of Negative Adjectives'] = negative
    d['Number of Positive Adjectives'] = positive
    d['Negative Adjectives'] = l_negative
    d['Positive Adjectives'] = l_positive
    if positive + negative == 0:
        d['Sentiment Score'] = 0
    else:
        d['Sentiment Score'] = (positive - negative)/(positive + negative)

    return d


df['adjectives']=df['nltk'].apply(goodbadadjectives)

df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,nltk,adjectives
0,A32T2H8150OJLU,B00000JBLH,ARH,"[3, 4]",I bought my first HPC in about or so and it s...,5,"A solid performer, and long time friend",1094169600,"09 3, 2004","{'MWETokenizer': ['I', 'bought', 'my', 'first'...","{'Total Adjectives': 17, 'Number of Negative A..."
1,A3MAFS04ZABRGO,B00000JBLH,"Let it Be ""Alan""","[7, 9]",WHY THIS BELATED REVIEW I feel very obliged to...,5,"Price of GOLD is up, so don't bury the golden ...",1197676800,"12 15, 2007","{'MWETokenizer': ['WHY', 'THIS', 'BELATED', 'R...","{'Total Adjectives': 58, 'Number of Negative A..."
2,A1F1A0QQP2XVH5,B00000JBLH,Mark B,"[3, 3]",I have an HP GX that has been kicking for more...,2,"Good functionality, but not durable like old HPs",1293840000,"01 1, 2011","{'MWETokenizer': ['I', 'have', 'an', 'HP', 'GX...","{'Total Adjectives': 8, 'Number of Negative Ad..."
3,A49R5DBXXQDE5,B00000JBLH,R. D Johnson,"[7, 8]",Ive started doing more finance stuff recently ...,5,One of the last of an almost extinct species,1145404800,"04 19, 2006","{'MWETokenizer': ['Ive', 'started', 'doing', '...","{'Total Adjectives': 43, 'Number of Negative A..."
4,A2XRMQA6PJ5ZJ8,B00000JBLH,Roger J. Buffington,"[0, 0]",For simple calculations and discounted cash fl...,5,Still the best,1375574400,"08 4, 2013","{'MWETokenizer': ['For', 'simple', 'calculatio...","{'Total Adjectives': 11, 'Number of Negative A..."


In [58]:
print(df.iloc[0, 9]['adjective'])
print(df.iloc[0, 10])

['first', 'difficult', 'many', 'disasterThe', 'powerful', 'other', 'good', 'solid', 'reliable', 'programmable', 'basic', 'many', 'manual', 'short', 'ready', 'old', 'useHP']
{'Total Adjectives': 17, 'Number of Negative Adjectives': 1, 'Number of Positive Adjectives': 16, 'Negative Adjectives': ['difficult'], 'Positive Adjectives': ['first', 'many', 'disasterThe', 'powerful', 'other', 'good', 'solid', 'reliable', 'programmable', 'basic', 'many', 'manual', 'short', 'ready', 'old', 'useHP'], 'Sentiment Score': 0.8823529411764706}
